# 8-1 Decision Tree
# language : Python
https://www.youtube.com/watch?v=jVh5NA9ERDA&list=PLqnslRFeH2Upcrywf-u2etjdxxkL8nl7E&index=8

## 0. 모델 기초

### Entropy
$ E = - \sum p(X) \cdot log_2(p(X))  , p(X) = \frac{\# x}{n}$

## 1. 모델 구성 

In [11]:
import numpy as np
from collections import Counter

def entropy(y):
     hist = np.bincount(y)
     ps = hist / len(y)
     return -np.sum([p * np.log2(p) for p in ps if p > 0])


class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None  # Node에 value가 있으면 True 반환
        


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None


    def fit(self, X, y):
        # grow tree
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)


    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedy search
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)


    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thres = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thres = threshold
        
        return split_idx, split_thres
    
    
    def _information_gain(self, y, X_column, split_thres):
        # parent E
        parent_entropy = entropy(y)
        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thres)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        # weighted avg child E
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # return ig
        ig = parent_entropy - child_entropy
        return ig


    def _split(self, X_column, split_thres):
        left_idxs = np.argwhere(X_column <= split_thres).flatten()
        right_idxs = np.argwhere(X_column > split_thres).flatten()
        return left_idxs, right_idxs


    def predict(self, X):
        # traverse tree
        return np.array([self._traverse_tree(x, self.root) for x in X])


    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)

    
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

## 2. sample data만들기

In [12]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print('Accuracy:', acc)

Accuracy: 0.9210526315789473


## 3. 추가학습

### np.bincount() 함수란?

In [13]:
import numpy as np

In [14]:
y = np.array(np.random.randint(1,4,10))
print(y)
print(np.bincount(y))

[2 3 1 2 3 1 3 2 2 3]
[0 2 4 4]


0부터 갯수를 세준다. 0이 하나도 없지만 0부터 세준다. 그렇다면 숫자가 커도 0부터 세줄까? 음수가 있으면 어떻게 될까?

In [15]:
y = np.array(np.random.randint(-3,3,10))
print(y)
print(np.bincount(y))

[ 0  0 -1 -2  1 -1 -2 -3 -2 -1]


ValueError: 'list' argument must have no negative elements

In [ ]:
y = np.array(np.random.randint(30,32,10))
print(y)
print(np.bincount(y))

[31 31 31 31 30 31 31 30 31 31]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 8]


### np.argwhere 이란?

In [23]:
a = np.array([3, 2, 1, 10, 9, 8, 4, 5, 6, 7])
print(a.max())
print(a.argmax())
print(np.argwhere(a==5))

10
3
[[7]]


In [32]:
a = np.array([1,1,1,1,1,1])
idx = np.argwhere(a==1)
print(idx)
print(idx.shape)
print(idx.flatten())
print(idx.flatten().shape)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
(6, 1)
[0 1 2 3 4 5]
(6,)
